In [1]:
# Importing Necessary libraries
import pyspark # Python API written in python to support Apache Spark

In [2]:
# Importing various libraries

import numpy as np # For creating n-dimensional array object
import seaborn as sns  # For making statistical graphics
import matplotlib.pyplot as plt  # Plotting the graphs
import pandas as pd # For creating array 

In [3]:
# Creating Spark Session

#Spark session creation for providing the single point entry to interact with underlying spark functionalities and allows programming-
# Spark with dataframe and dataset APIs

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark on European football") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Creating Spark Context

# importing the data type
from pyspark.sql.types import *

# Creating Spark context.It is the entry gate of Apache Spark functionalities.
sc = spark.sparkContext

In [5]:
# Reading the CSV file having details of all football incidents

# Read the CSV file name 'Events.csv' having detail of events such as shot_place, shot_outcome ,is_game etc.
# These details will be further used in visualization.

eventsDf = spark.read.options(header="true",\
                              inferSchema="true",\
                              nullValue="NA",\
                              mode="failfast")\
                             .csv("/FileStore/tables/events.csv")

In [6]:
# Exploring and Storing Datasets

eventsDf.show(5)

+---------+---------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+----------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
 id_odsp| id_event|sort_order|time| text|event_type|event_type2|side| event_team| opponent| player| player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|
+---------+---------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+----------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
UFot0hit/|UFot0hit1| 1| 2|Attempt missed. M...| 1| 12| 2| Hamburg SV|Borussia Dortmund| mladen petric| gokhan tore| null| null| 6| 2| 0| 9| 2| 1| 1| 0|
UFot0hit/|UFot0hit2| 2| 4|Corner, Borussia...| 2| null| 1|Borussia Dortmund| Hamburg SV|dennis diekmeier|dennis diekmeier| null| null| null| null| 0| null| null| 0| null| 0|
UFot0hit/|UFot0hit3| 3| 4|Corner, Borussia...| 2| null| 1|Borussia Dortmund| Hamburg SV|heiko westermann|heiko westermann| null| null| null| null| 0| null| null| 0| null| 0|
UFot0hit/|UFot0hit4| 4| 7|Foul by Sven Bend...| 3| null| 1|Borussia Dortmund| Hamburg SV| sven bender| null| null| null| null| null| 0| null| null| 0| null| 0|
UFot0hit/|UFot0hit5| 5| 7|Gokhan Tore (Hamb...| 8| null| 2| Hamburg SV|Borussia Dortmund| gokhan tore| null| null| null| null| null| 0| 2| null| 0| null| 0|
+---------+---------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+----------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
only showing top 5 rows

In [7]:
# Counting the number of Rows and columns in dataset.

print('Number of rows', eventsDf.count(), ",and", len(eventsDf.columns), 'columns' )

Number of rows 941009 ,and 22 columns

In [8]:
# Printing the data schema

eventsDf.printSchema()

root
-- id_odsp: string (nullable = true)
-- id_event: string (nullable = true)
-- sort_order: integer (nullable = true)
-- time: integer (nullable = true)
-- text: string (nullable = true)
-- event_type: integer (nullable = true)
-- event_type2: integer (nullable = true)
-- side: integer (nullable = true)
-- event_team: string (nullable = true)
-- opponent: string (nullable = true)
-- player: string (nullable = true)
-- player2: string (nullable = true)
-- player_in: string (nullable = true)
-- player_out: string (nullable = true)
-- shot_place: integer (nullable = true)
-- shot_outcome: integer (nullable = true)
-- is_goal: integer (nullable = true)
-- location: integer (nullable = true)
-- bodypart: integer (nullable = true)
-- assist_method: integer (nullable = true)
-- situation: integer (nullable = true)
-- fast_break: integer (nullable = true)

In [9]:
# Partitions in dataset

# Getting number of partitions in event dataset.
# RDDs are immutable, we can not change the partition of RDDs. 
# New RDDs can be create with the desired number of partitions

eventsDf.rdd.getNumPartitions()

Out[11]: 8

In [10]:
eventsDf.show(1)

+---------+---------+----------+----+--------------------+----------+-----------+----+----------+-----------------+-------------+-----------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
 id_odsp| id_event|sort_order|time| text|event_type|event_type2|side|event_team| opponent| player| player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|
+---------+---------+----------+----+--------------------+----------+-----------+----+----------+-----------------+-------------+-----------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
UFot0hit/|UFot0hit1| 1| 2|Attempt missed. M...| 1| 12| 2|Hamburg SV|Borussia Dortmund|mladen petric|gokhan tore| null| null| 6| 2| 0| 9| 2| 1| 1| 0|
+---------+---------+----------+----+--------------------+----------+-----------+----+----------+-----------------+-------------+-----------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
only showing top 1 row

In [11]:
# Checking null values

# Verifying that null values are present in the required variables or not.

from pyspark.sql.functions import isnan, when, count, col

# Creating temporary data files. Transfering all columns in temporary data file.

tempDF = eventsDf[['player', 'event_team', 'opponent', 'event_type', 'event_type2', 'shot_place', 
                   'shot_outcome', 'location', 'bodypart', 'assist_method', 'situation']]

# Doing alliasing of temporary schema 

tempDF.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in tempDF.columns]).show()

+------+----------+--------+----------+-----------+----------+------------+--------+--------+-------------+---------+
player|event_team|opponent|event_type|event_type2|shot_place|shot_outcome|location|bodypart|assist_method|situation|
+------+----------+--------+----------+-----------+----------+------------+--------+--------+-------------+---------+
 60736| 0| 0| 0| 726716| 713550| 712511| 473942| 711824| 0| 711872|
+------+----------+--------+----------+-----------+----------+------------+--------+--------+-------------+---------+

In [12]:
# Managing Null values

# Standrd feature for filling 'NA' values.

eventsDf = eventsDf.na.fill({'player': 'NA', 'event_team': 'NA', 'opponent': 'NA', 'event_type': 'NA', 'event_type2': 'NA', 'shot_place': 'NA',
                             'shot_outcome': 'NA', 'location': 'NA', 'bodypart': 'NA','assist_method': 'NA', 'situation': 'NA'}) 


In [13]:
# Importing another CSV file

# Reading the Game data set.

gameDf = spark.read.options(header="true",\
                              inferSchema="true",\
                              nullValue="NA",\
                              mode="failfast")\
                             .csv("/FileStore/tables/ginf.csv")

In [14]:
# Repeating Above procedure on Game CSV file.

In [15]:
# Printing the CSV file

gameDf.printSchema()

root
-- id_odsp: string (nullable = true)
-- link_odsp: string (nullable = true)
-- adv_stats: boolean (nullable = true)
-- date: timestamp (nullable = true)
-- league: string (nullable = true)
-- season: integer (nullable = true)
-- country: string (nullable = true)
-- ht: string (nullable = true)
-- at: string (nullable = true)
-- fthg: integer (nullable = true)
-- ftag: integer (nullable = true)
-- odd_h: double (nullable = true)
-- odd_d: double (nullable = true)
-- odd_a: double (nullable = true)
-- odd_over: double (nullable = true)
-- odd_under: double (nullable = true)
-- odd_bts: double (nullable = true)
-- odd_bts_n: double (nullable = true)

In [16]:
# Getting number of partitions in event dataset.
# RDDs are immutable, we can not change the partition of RDDs. 
# New RDDs can be create with the desired number of partitions


gameDf.rdd.getNumPartitions()

Out[17]: 1

In [17]:
# Printing the Game data frame

gameDf.show(5)

+---------+--------------------+---------+-------------------+------+------+-------+-------------------+--------------+----+----+-----+-----+-----+--------+---------+-------+---------+
 id_odsp| link_odsp|adv_stats| date|league|season|country| ht| at|fthg|ftag|odd_h|odd_d|odd_a|odd_over|odd_under|odd_bts|odd_bts_n|
+---------+--------------------+---------+-------------------+------+------+-------+-------------------+--------------+----+----+-----+-----+-----+--------+---------+-------+---------+
UFot0hit/|/soccer/germany/b...| true|2011-08-05 00:00:00| D1| 2012|germany| Borussia Dortmund| Hamburg SV| 3| 1| 1.56| 4.41| 7.42| null| null| null| null|
Aw5DflLH/|/soccer/germany/b...| true|2011-08-06 00:00:00| D1| 2012|germany| FC Augsburg| SC Freiburg| 2| 2| 2.36| 3.6| 3.4| null| null| null| null|
bkjpaC6n/|/soccer/germany/b...| true|2011-08-06 00:00:00| D1| 2012|germany| Werder Bremen|Kaiserslautern| 2| 0| 1.83| 4.2| 4.8| null| null| null| null|
CzPV312a/|/soccer/france/li...| true|2011-08-06 00:00:00| F1| 2012| france|Paris Saint-Germain| Lorient| 0| 1| 1.55| 4.5| 9.4| null| null| null| null|
GUOdmtII/|/soccer/france/li...| true|2011-08-06 00:00:00| F1| 2012| france| Caen| Valenciennes| 1| 0| 2.5| 3.4| 3.45| null| null| null| null|
+---------+--------------------+---------+-------------------+------+------+-------+-------------------+--------------+----+----+-----+-----+-----+--------+---------+-------+---------+
only showing top 5 rows

In [18]:
# Taking Desired Columns 

# Taking Desired Columns

gameDf = gameDf[['id_odsp', 'country', 'adv_stats', 'date', 'league', 'season']]

In [19]:
gameDf.show(5)

+---------+-------+---------+-------------------+------+------+
 id_odsp|country|adv_stats| date|league|season|
+---------+-------+---------+-------------------+------+------+
UFot0hit/|germany| true|2011-08-05 00:00:00| D1| 2012|
Aw5DflLH/|germany| true|2011-08-06 00:00:00| D1| 2012|
bkjpaC6n/|germany| true|2011-08-06 00:00:00| D1| 2012|
CzPV312a/| france| true|2011-08-06 00:00:00| F1| 2012|
GUOdmtII/| france| true|2011-08-06 00:00:00| F1| 2012|
+---------+-------+---------+-------------------+------+------+
only showing top 5 rows

In [20]:
# Verifying if the data set has null values to treat
gameDf.select([count(when(col(c).isNull(), c)).alias(c) for c in gameDf.columns]).show()

+-------+-------+---------+----+------+------+
id_odsp|country|adv_stats|date|league|season|
+-------+-------+---------+----+------+------+
 0| 0| 0| 0| 0| 0|
+-------+-------+---------+----+------+------+

In [21]:
# Dictionary.txt file :- 
# event_type
# 0	Announcement
# 1	Attempt
# 2	Corner
# 3	Foul
# 4	Yellow card
# 5	Second yellow card
# 6	Red card
# 7	Substitution
# 8	Free kick won
# 9	Offside
# 10	Hand ball
# 11	Penalty conceded

# In below code Dictionary.txt file had been converted into Python Dictionaries.


In [22]:
# Processing data for incorporating the text file

#Converting Text file into Python Dictionaries

# Mapping Various events of football
evtTypeMap = {0:'Announcement', 1:'Attempt', 2:'Corner', 3:'Foul', 4:'Yellow card', 5:'Second yellow card', 6:'Red card', 7:'Substitution', 8:'Free kick won', 9:'Offside', 10:'Hand ball', 11:'Penalty conceded'}

evtTyp2Map = {12:'Key Pass', 13:'Failed through ball', 14:'Sending off', 15:'Own goal'}

sideMap = {1:'Home', 2:'Away'}

#  Type of football kick
shotPlaceMap = {1:'Bit too high', 2:'Blocked', 3:'Bottom left corner', 4:'Bottom right corner', 5:'Centre of the goal', 6:'High and wide', 7:'Hits the bar', 8:'Misses to the left', 9:'Misses to the right', 10:'Too high', 11:'Top centre of the goal', 12:'Top left corner', 13:'Top right corner'}

# Mapping of type of shot
shotOutcomeMap = {1:'On target', 2:'Off target', 3:'Blocked', 4:'Hit the bar'}

# Location form where football had been kicked 
locationMap = {1:'Attacking half', 2:'Defensive half', 3:'Centre of the box', 4:'Left wing', 5:'Right wing', 6:'Difficult angle and long range', 7:'Difficult angle on the left', 8:'Difficult angle on the right', 9:'Left side of the box', 10:'Left side of the six yard box', 11:'Right side of the box', 12:'Right side of the six yard box', 13:'Very close range', 14:'Penalty spot', 15:'Outside the box', 16:'Long range', 17:'More than 35 yards', 18:'More than 40 yards', 19:'Not recorded'}

# Which body part had been used, while kicking the football
bodyPartMap = {1:'Right foot', 2:'Left foot', 3:'Head'}

# Mapping of type of football kick
assistMethodMap = {0:'None', 1:'Pass', 2:'Cross', 3:'Headed pass', 4:'Through ball'}

# Mapping of Different football playgorund situations
situationMap = {1:'Open play', 2:'Set piece', 3:'Corner', 4:'Free kick'}

# Given Code names to the countries that are taking participation in Football leagus matches
countryCodeMap = {'germany':'DEU', 'france':'FRA', 'england':'GBR', 'spain':'ESP', 'italy':'ITA'}

In [23]:
# Counting Number of Event types from 'eventsDf' file

eventsDf.groupBy("event_type").count().show()

+----------+------+
event_type| count|
+----------+------+
 1|229135|
 6| 1152|
 3|232925|
 5| 100|
 9| 43476|
 4| 39911|
 8|237932|
 7| 51738|
 10| 10730|
 11| 2706|
 2| 91204|
+----------+------+

In [24]:
# Creating User Defined Functions(UDF)

# Defining the UDF for mapping keys to value in dictionary
# UDFs work in a similar way as the pandas . map()

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def translateKeys(mapping):
    def translateKeys_(col):
        return mapping.get(col)
    return udf(translateKeys_, StringType())

In [25]:
#Creating country code for quick country column access and country name errors 

gameDf = gameDf.withColumn("country_code", translateKeys(countryCodeMap)("country"))
gameDf['id_odsp','country','country_code']

gameDf.show(5)

+---------+-------+---------+-------------------+------+------+------------+
 id_odsp|country|adv_stats| date|league|season|country_code|
+---------+-------+---------+-------------------+------+------+------------+
UFot0hit/|germany| true|2011-08-05 00:00:00| D1| 2012| DEU|
Aw5DflLH/|germany| true|2011-08-06 00:00:00| D1| 2012| DEU|
bkjpaC6n/|germany| true|2011-08-06 00:00:00| D1| 2012| DEU|
CzPV312a/| france| true|2011-08-06 00:00:00| F1| 2012| FRA|
GUOdmtII/| france| true|2011-08-06 00:00:00| F1| 2012| FRA|
+---------+-------+---------+-------------------+------+------+------------+
only showing top 5 rows

In [26]:
## Add additional columns with the corresponding dict type name
# Performing a lookups using UDFs and joining Dataframes

eventsDf = (
             eventsDf.
             withColumn("event_type_str", translateKeys(evtTypeMap)("event_type")).
             withColumn("event_type2_str", translateKeys(evtTyp2Map)("event_type2")).
             withColumn("side_str", translateKeys(sideMap)("side")).
             withColumn("shot_place_str", translateKeys(shotPlaceMap)("shot_place")).
             withColumn("shot_outcome_str", translateKeys(shotOutcomeMap)("shot_outcome")).
             withColumn("location_str", translateKeys(locationMap)("location")).
             withColumn("bodypart_str", translateKeys(bodyPartMap)("bodypart")).
             withColumn("assist_method_str", translateKeys(assistMethodMap)("assist_method")).
             withColumn("situation_str", translateKeys(situationMap)("situation"))
           )

In [27]:
# See if all column for corresponding dictionary mapping generated with string sort.

# eventsDf.show(5)
eventsDf.printSchema()

root
-- id_odsp: string (nullable = true)
-- id_event: string (nullable = true)
-- sort_order: integer (nullable = true)
-- time: integer (nullable = true)
-- text: string (nullable = true)
-- event_type: integer (nullable = true)
-- event_type2: integer (nullable = true)
-- side: integer (nullable = true)
-- event_team: string (nullable = false)
-- opponent: string (nullable = false)
-- player: string (nullable = false)
-- player2: string (nullable = true)
-- player_in: string (nullable = true)
-- player_out: string (nullable = true)
-- shot_place: integer (nullable = true)
-- shot_outcome: integer (nullable = true)
-- is_goal: integer (nullable = true)
-- location: integer (nullable = true)
-- bodypart: integer (nullable = true)
-- assist_method: integer (nullable = true)
-- situation: integer (nullable = true)
-- fast_break: integer (nullable = true)
-- event_type_str: string (nullable = true)
-- event_type2_str: string (nullable = true)
-- side_str: string (nullable = true)
-- shot_place_str: string (nullable = true)
-- shot_outcome_str: string (nullable = true)
-- location_str: string (nullable = true)
-- bodypart_str: string (nullable = true)
-- assist_method_str: string (nullable = true)
-- situation_str: string (nullable = true)

In [28]:
# Verifying whether core values properly expressed or not

eventsDf.groupBy("event_type_str").count().show()

+------------------+------+
 event_type_str| count|
+------------------+------+
 Hand ball| 10730|
 Red card| 1152|
 Foul|232925|
 Yellow card| 39911|
 Attempt|229135|
 Substitution| 51738|
 Offside| 43476|
 Penalty conceded| 2706|
 Free kick won|237932|
Second yellow card| 100|
 Corner| 91204|
+------------------+------+

In [29]:
# Creating final data frame by adding Game and Event data frame.

finalDf = (
  eventsDf.join(gameDf, eventsDf.id_odsp == gameDf.id_odsp, 'inner').
  select(eventsDf.id_odsp, eventsDf.id_event, eventsDf.sort_order, eventsDf.time, eventsDf.event_type, 
         eventsDf.event_type_str, eventsDf.event_type2, eventsDf.event_type2_str, eventsDf.side, 
         eventsDf.side_str, eventsDf.event_team, eventsDf.opponent, eventsDf.player, eventsDf.player2, 
         eventsDf.player_in, eventsDf.player_out, eventsDf.shot_place, eventsDf.shot_place_str, 
         eventsDf.shot_outcome, eventsDf.shot_outcome_str, eventsDf.is_goal, eventsDf.location, 
         eventsDf.location_str, eventsDf.bodypart, eventsDf.bodypart_str, eventsDf.assist_method,
         eventsDf.assist_method_str, eventsDf.situation, eventsDf.situation_str, gameDf.date, gameDf.league,
         gameDf.season, gameDf.country, gameDf.country_code, gameDf.adv_stats)
)

In [30]:
# Checking the final structure for all columns

finalDf.printSchema()

root
-- id_odsp: string (nullable = true)
-- id_event: string (nullable = true)
-- sort_order: integer (nullable = true)
-- time: integer (nullable = true)
-- event_type: integer (nullable = true)
-- event_type_str: string (nullable = true)
-- event_type2: integer (nullable = true)
-- event_type2_str: string (nullable = true)
-- side: integer (nullable = true)
-- side_str: string (nullable = true)
-- event_team: string (nullable = false)
-- opponent: string (nullable = false)
-- player: string (nullable = false)
-- player2: string (nullable = true)
-- player_in: string (nullable = true)
-- player_out: string (nullable = true)
-- shot_place: integer (nullable = true)
-- shot_place_str: string (nullable = true)
-- shot_outcome: integer (nullable = true)
-- shot_outcome_str: string (nullable = true)
-- is_goal: integer (nullable = true)
-- location: integer (nullable = true)
-- location_str: string (nullable = true)
-- bodypart: integer (nullable = true)
-- bodypart_str: string (nullable = true)
-- assist_method: integer (nullable = true)
-- assist_method_str: string (nullable = true)
-- situation: integer (nullable = true)
-- situation_str: string (nullable = true)
-- date: timestamp (nullable = true)
-- league: string (nullable = true)
-- season: integer (nullable = true)
-- country: string (nullable = true)
-- country_code: string (nullable = true)
-- adv_stats: boolean (nullable = true)

In [31]:
# Creating Final Data File using Quantile Discretizer

#QuantileDiscretizer takes a column with continuous features and outputs a column.
from pyspark.ml.feature import QuantileDiscretizer

# The bin ranges are chosen based on quantiles, that is, each bin will have the same number of observations.
finalDf = QuantileDiscretizer(numBuckets=10, inputCol="time", outputCol="time_bin").fit(finalDf).transform(finalDf)

In [32]:
# Checking the Null values if Dataset has any.

finalDf.select([count(when(col(c).isNull(), c)).alias(c) for c in finalDf.columns]).show()

+-------+--------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------+-------+---------+----------+----------+--------------+------------+----------------+-------+--------+------------+--------+------------+-------------+-----------------+---------+-------------+----+------+------+-------+------------+---------+--------+
id_odsp|id_event|sort_order|time|event_type|event_type_str|event_type2|event_type2_str|side|side_str|event_team|opponent|player|player2|player_in|player_out|shot_place|shot_place_str|shot_outcome|shot_outcome_str|is_goal|location|location_str|bodypart|bodypart_str|assist_method|assist_method_str|situation|situation_str|date|league|season|country|country_code|adv_stats|time_bin|
+-------+--------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------+-------+---------+----------+----------+--------------+------------+----------------+-------+--------+------------+--------+------------+-------------+-----------------+---------+-------------+----+------+------+-------+------------+---------+--------+
 0| 0| 0| 0| 0| 0| 726716| 726716| 0| 0| 0| 0| 0| 649696| 889294| 889271| 713550| 713550| 712511| 712511| 0| 473942| 473942| 711824| 711824| 0| 0| 711872| 711872| 0| 0| 0| 0| 0| 0| 0|
+-------+--------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------+-------+---------+----------+----------+--------------+------------+----------------+-------+--------+------------+--------+------------+-------------+-----------------+---------+-------------+----+------+------+-------+------------+---------+--------+

In [33]:
# Handling the null values 

finalDf = finalDf.na.fill({'player': 'NA', 'event_team': 'NA', 'opponent': 'NA', 
                             'event_type_str': 'NA', 'event_type2_str': 'NA', 'shot_place_str': 'NA', 
                             'shot_outcome_str': 'NA', 'location_str': 'NA', 'bodypart_str': 'NA', 
                             'assist_method_str': 'NA', 'situation_str': 'NA'})

In [34]:
finalDf.rdd.getNumPartitions()

Out[36]: 8

In [36]:
# Data Visualization and Analysis using spark SQL for processing of structured data

#Research Question 1 :- What are the spots on the ground with the highest number of assists?

# Accessing the SQL Temporary Dataframe
finalDf.createOrReplaceTempView("soccer")

# Using SQL query for getting the total number of Goals at Particular Shot Place  

que1 = spark.sql("SELECT CASE WHEN shot_place_str == 'NA' THEN 'Unknown' ELSE shot_place_str END shot_place,\
                    COUNT(1) AS TOT_GOALS \
                    FROM soccer WHERE is_goal = 1 \
                    GROUP BY shot_place_str \
                    ORDER BY TOT_GOALS DESC")
que1.show()

+-------------------+---------+
 shot_place|TOT_GOALS|
+-------------------+---------+
 Bottom left corner| 7212|
Bottom right corner| 6932|
 Centre of the goal| 4446|
 Top right corner| 2157|
 Top left corner| 2023|
 Unknown| 1676|
+-------------------+---------+

In [37]:
# By looking at above data, we can clearly infer that Maixmum number of goals had been done from 'Bottom Left corner'

In [38]:
display(que1)

shot_place,TOT_GOALS
Bottom left corner,7212
Bottom right corner,6932
Centre of the goal,4446
Top right corner,2157
Top left corner,2023
Unknown,1676


In [39]:

# RQ 2 :-In all competitions, which nations have recorded maximum number of goals?

# SQL query for selecting countries that had done maximum Goals 

que2 = spark.sql("SELECT country, COUNT(1) AS TOT_GOALS FROM soccer WHERE is_goal = 1 GROUP BY country\
                    ORDER BY 2 DESC")
que2.show()

+-------+---------+
country|TOT_GOALS|
+-------+---------+
 spain| 5583|
 italy| 5491|
 france| 5199|
germany| 4621|
england| 3552|
+-------+---------+

In [40]:
# Plot above result in pie chart
display(que2)

country,TOT_GOALS
spain,5583
italy,5491
france,5199
germany,4621
england,3552


In [41]:
# By Looking at pie chart and table , we can say that 'Spain' had done the maximum Number of Goals in league Matches

In [42]:
# RQ 3 :- What are the few minutes with maximum goals?

# SQL query for selecting time and maximum number of goals. We had set the limit to 10 records only.

que3 = spark.sql("SELECT time, count(1) as num_of_goals FROM soccer WHERE is_goal = 1 GROUP BY time ORDER BY 2 DESC LIMIT 10")
                 
que3.show()

+----+------------+
time|num_of_goals|
+----+------------+
 90| 1244|
 45| 594|
 56| 339|
 72| 308|
 50| 307|
 86| 306|
 73| 306|
 88| 305|
 61| 304|
 55| 300|
+----+------------+

In [43]:
# Plot above result
res3 = que3.toPandas()
plot3 = res3.plot.bar(x='time', y='num_of_goals', figsize=(20,5))
display(plot3)

In [44]:
# Maximum number of goals had been done in 90th minute followed by 45th minute.

In [45]:
# RQ 4 :-  Possible time frame (minutes) for getting the yellow card as many as possible? 

# Selecting the columns Time and Num_of_yellow_cards for getting the desired results

que4 = spark.sql("SELECT time, count(1) as num_of_yellow_cards FROM soccer \
                WHERE event_type_str == 'Yellow card' OR event_type_str == 'Second yellow card' \
                GROUP BY time ORDER BY 2 DESC LIMIT 10")
                 
que4.show()

+----+-------------------+
time|num_of_yellow_cards|
+----+-------------------+
 90| 2784|
 45| 1106|
 88| 686|
 89| 650|
 87| 624|
 86| 618|
 84| 593|
 85| 592|
 80| 588|
 83| 585|
+----+-------------------+

In [46]:
# Plotting the bar graph using above results

res4 = que4.toPandas()
plot4 = res4.plot.bar(x='time', y='num_of_yellow_cards', figsize=(18,7))
display(plot4)

In [47]:
# By looking at Graph, we can say that Maximum number of Yellow cards had been given in 90th Minute.

# We can also illustrate that Maximum number of goals and maximum number of yellow cards had been given on same time frame i.e. 90th Minute. 


In [48]:
# RQ 5 :- In which minutes have given the maximum number of red cards?
# Selecting the Time and Maximum number of Red cards columns from Final Data file.  

que5 = spark.sql("SELECT time, count(1) as num_of_red_card FROM soccer \
                WHERE event_type_str == 'Red card' \
                GROUP BY time ORDER BY 2 DESC LIMIT 10")
                 
que5.show()

+----+---------------+
time|num_of_red_card|
+----+---------------+
 90| 165|
 82| 29|
 86| 28|
 45| 27|
 83| 25|
 64| 25|
 81| 24|
 72| 22|
 80| 22|
 85| 21|
+----+---------------+

In [49]:
res5 = que5.toPandas()

plot5 = res5.plot.bar(x='time', y='num_of_red_card', figsize=(18,7))

display(plot5)

In [50]:
# In 90th Minute maximum number of red card had been given and it is followed by 82th minute for maximum red cards.

In [51]:
# Machine learning Algorithms
#  We will use GTB classifier machine learning algorithm.

#  GBT is a supervised machine learning algorithm for classiﬁcation. This constructs the model in a phase-by-phase way like many other boosting techniques #  and generalizes it through optimization.

In [52]:
# Importing necessary libraries for  GBT Classification model

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [53]:
# Categorical features to single binary vectors

# Build a categorical characteristic list
categFeatures = ["event_type_str", "event_team", "shot_place_str", "location_str", "assist_method_str", "situation_str", "country_code"]

# Encode the string cols for categorical mark indices
stringIndexers = [StringIndexer().setInputCol(baseFeature).setOutputCol(baseFeature + "_idx") for baseFeature in categFeatures]

# Used to transform indexes to binary vectors. 
encoders = [OneHotEncoder().setInputCol(baseFeature + "_idx").setOutputCol(baseFeature + "_vec") for baseFeature in categFeatures]


In [54]:
# Combine all columns into a single feature vector
featureAssembler = VectorAssembler()
featureAssembler.setInputCols([baseFeature + "_vec" for baseFeature in categFeatures])
featureAssembler.setOutputCol("features")

Out[40]: VectorAssembler_665eb1412ec0

In [55]:
# Creating pipeline and splitting the data into training and testing data

from pyspark.ml.classification import GBTClassifier


gbtClassifier = GBTClassifier(labelCol="is_goal", featuresCol="features", maxDepth=5, maxIter=20)

pipelineStages = stringIndexers + encoders + [featureAssembler, gbtClassifier]
pipeline = Pipeline(stages=pipelineStages)


# (trainingData, testData) = joinedDf.randomSplit([0.75, 0.30])
# eventsDf
(trainingData, testData) = finalDf.randomSplit([0.70, 0.30])

In [56]:
# model = pipeline.fit(trainingData)
#Is_goal in events_df

model = pipeline.fit(trainingData)

In [57]:
# Validate the model on test data, display predictions
predictions = model.transform(testData)
predictions.select("prediction", "is_goal", "features")
predictions.show(5)

+---------+-----------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------------------+---------------+---------+----------+----------+--------------+------------+----------------+-------+--------+-----------------+--------+------------+-------------+-----------------+---------+-------------+-------------------+------+------+-------+------------+---------+--------+------------------+--------------+------------------+----------------+---------------------+-----------------+----------------+------------------+---------------+------------------+----------------+---------------------+-----------------+----------------+--------------------+--------------------+--------------------+----------+
 id_odsp| id_event|sort_order|time|event_type|event_type_str|event_type2|event_type2_str|side|side_str|event_team|opponent| player| player2|player_in|player_out|shot_place|shot_place_str|shot_outcome|shot_outcome_str|is_goal|location| location_str|bodypart|bodypart_str|assist_method|assist_method_str|situation|situation_str| date|league|season|country|country_code|adv_stats|time_bin|event_type_str_idx|event_team_idx|shot_place_str_idx|location_str_idx|assist_method_str_idx|situation_str_idx|country_code_idx|event_type_str_vec| event_team_vec|shot_place_str_vec|location_str_vec|assist_method_str_vec|situation_str_vec|country_code_vec| features| rawPrediction| probability|prediction|
+---------+-----------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------------------+---------------+---------+----------+----------+--------------+------------+----------------+-------+--------+-----------------+--------+------------+-------------+-----------------+---------+-------------+-------------------+------+------+-------+------------+---------+--------+------------------+--------------+------------------+----------------+---------------------+-----------------+----------------+------------------+---------------+------------------+----------------+---------------------+-----------------+----------------+--------------------+--------------------+--------------------+----------+
00LMl81F/|00LMl81F103| 103| 79| 3| Foul| null| NA| 2| Away| AC Milan| AS Roma| antonio nocerino| null| null| null| null| NA| null| NA| 0| null| NA| null| NA| 0| None| null| NA|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 1.0| 3.0| 0.0| 0.0| 0.0| 0.0| 0.0| (10,[1],[1.0])|(141,[3],[1.0])| (13,[0],[1.0])| (19,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])|(195,[1,13,151,16...|[1.54037586576742...|[0.95609175335075...| 0.0|
00LMl81F/|00LMl81F104| 104| 79| 8| Free kick won| null| NA| 1| Home| AS Roma|AC Milan| daniele de rossi| null| null| null| null| NA| null| NA| 0| 1| Attacking half| null| NA| 0| None| null| NA|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 0.0| 6.0| 0.0| 4.0| 0.0| 0.0| 0.0| (10,[0],[1.0])|(141,[6],[1.0])| (13,[0],[1.0])| (19,[4],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])|(195,[0,16,151,16...|[1.54037586576745...|[0.95609175335075...| 0.0|
00LMl81F/|00LMl81F105| 105| 80| 1| Attempt| 12| Key Pass| 1| Home| AS Roma|AC Milan| nicolas burdisso| bojan| null| null| 2| Blocked| 3| Blocked| 0| 3|Centre of the box| 2| Left foot| 3| Headed pass| 1| Open play|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 2.0| 6.0| 1.0| 3.0| 3.0| 1.0| 0.0| (10,[2],[1.0])|(141,[6],[1.0])| (13,[1],[1.0])| (19,[3],[1.0])| (4,[3],[1.0])| (4,[1],[1.0])| (4,[0],[1.0])|(195,[2,16,152,16...|[1.46750636968064...|[0.94955034960809...| 0.0|
00LMl81F/|00LMl81F107| 107| 81| 1| Attempt| 12| Key Pass| 2| Away| AC Milan| AS Roma|zlatan ibrahimovic|antonio cassano| null| null| 6| High and wide| 2| Off target| 0| 3|Centre of the box| 1| Right foot| 2| Cross| 1| Open play|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 2.0| 3.0| 7.0| 3.0| 2.0| 1.0| 0.0| (10,[2],[1.0])|(141,[3],[1.0])| (13,[7],[1.0])| (19,[3],[1.0])| (4,[2],[1.0])| (4,[1],[1.0])| (4,[0]

In [58]:
# Evaluating the performance of model

evaluator = BinaryClassificationEvaluator(
    labelCol="is_goal", rawPredictionCol="prediction")
evaluator.evaluate(predictions)

Out[44]: 0.7764384692288578

In [59]:
# Logistic Regression

from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'is_goal', maxIter=10)

pipelineStages1 = stringIndexers + encoders + [featureAssembler, lr]
pipeline1 = Pipeline(stages=pipelineStages)


In [60]:
# Splitting the data into training and testing

(trainingData, testData) = finalDf.randomSplit([0.70, 0.30])

In [61]:
# Creainng a lrmodel

lrModel = pipeline1.fit(trainingData)

In [62]:
predictions1 = lrModel.transform(testData)
predictions1.select("prediction", "is_goal", "features")
predictions1.show(5)

+---------+-----------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------------------+---------------+---------+----------+----------+--------------+------------+----------------+-------+--------+-----------------+--------+------------+-------------+-----------------+---------+-------------+-------------------+------+------+-------+------------+---------+--------+------------------+--------------+------------------+----------------+---------------------+-----------------+----------------+------------------+---------------+------------------+----------------+---------------------+-----------------+----------------+--------------------+--------------------+--------------------+----------+
 id_odsp| id_event|sort_order|time|event_type|event_type_str|event_type2|event_type2_str|side|side_str|event_team|opponent| player| player2|player_in|player_out|shot_place|shot_place_str|shot_outcome|shot_outcome_str|is_goal|location| location_str|bodypart|bodypart_str|assist_method|assist_method_str|situation|situation_str| date|league|season|country|country_code|adv_stats|time_bin|event_type_str_idx|event_team_idx|shot_place_str_idx|location_str_idx|assist_method_str_idx|situation_str_idx|country_code_idx|event_type_str_vec| event_team_vec|shot_place_str_vec|location_str_vec|assist_method_str_vec|situation_str_vec|country_code_vec| features| rawPrediction| probability|prediction|
+---------+-----------+----------+----+----------+--------------+-----------+---------------+----+--------+----------+--------+------------------+---------------+---------+----------+----------+--------------+------------+----------------+-------+--------+-----------------+--------+------------+-------------+-----------------+---------+-------------+-------------------+------+------+-------+------------+---------+--------+------------------+--------------+------------------+----------------+---------------------+-----------------+----------------+------------------+---------------+------------------+----------------+---------------------+-----------------+----------------+--------------------+--------------------+--------------------+----------+
00LMl81F/|00LMl81F103| 103| 79| 3| Foul| null| NA| 2| Away| AC Milan| AS Roma| antonio nocerino| null| null| null| null| NA| null| NA| 0| null| NA| null| NA| 0| None| null| NA|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 1.0| 3.0| 0.0| 0.0| 0.0| 0.0| 0.0| (10,[1],[1.0])|(141,[3],[1.0])| (13,[0],[1.0])| (19,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])|(195,[1,13,151,16...|[1.54043538174671...|[0.95609675007830...| 0.0|
00LMl81F/|00LMl81F104| 104| 79| 8| Free kick won| null| NA| 1| Home| AS Roma|AC Milan| daniele de rossi| null| null| null| null| NA| null| NA| 0| 1| Attacking half| null| NA| 0| None| null| NA|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 0.0| 4.0| 0.0| 4.0| 0.0| 0.0| 0.0| (10,[0],[1.0])|(141,[4],[1.0])| (13,[0],[1.0])| (19,[4],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])| (4,[0],[1.0])|(195,[0,14,151,16...|[1.54043538174674...|[0.95609675007830...| 0.0|
00LMl81F/|00LMl81F105| 105| 80| 1| Attempt| 12| Key Pass| 1| Home| AS Roma|AC Milan| nicolas burdisso| bojan| null| null| 2| Blocked| 3| Blocked| 0| 3|Centre of the box| 2| Left foot| 3| Headed pass| 1| Open play|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 2.0| 4.0| 1.0| 3.0| 3.0| 1.0| 0.0| (10,[2],[1.0])|(141,[4],[1.0])| (13,[1],[1.0])| (19,[3],[1.0])| (4,[3],[1.0])| (4,[1],[1.0])| (4,[0],[1.0])|(195,[2,14,152,16...|[1.46307482034309...|[0.94912407177201...| 0.0|
00LMl81F/|00LMl81F107| 107| 81| 1| Attempt| 12| Key Pass| 2| Away| AC Milan| AS Roma|zlatan ibrahimovic|antonio cassano| null| null| 6| High and wide| 2| Off target| 0| 3|Centre of the box| 1| Right foot| 2| Cross| 1| Open play|2011-10-29 00:00:00| I1| 2012| italy| ITA| true| 8.0| 2.0| 3.0| 7.0| 3.0| 2.0| 1.0| 0.0| (10,[2],[1.0])|(141,[3],[1.0])| (13,[7],[1.0])| (19,[3],[1.0])| (4,[2],[1.0])| (4,[1],[1.0])| (4,[0]

In [63]:
# Evaluating the performance of logistic regression model
evaluator = BinaryClassificationEvaluator(
    labelCol="is_goal", rawPredictionCol="prediction")
evaluator.evaluate(predictions1)

Out[52]: 0.7655103841488528